In [ ]:
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import numpy as np

In [ ]:
def load(m):
    with open(f'/home/nahappy15/accepTAble/sample_stat_gaussian_noise_{m}.pkl', 'rb') as f:
        c_list = pickle.load(f)
    return m, c_list

In [ ]:
method_list = ['noadapt', 'eata', 'eata_aug', 'sar', 'sar_aug','deyo', 'deyo_aug',]
data = {}
for m in method_list:
    k, v = load(m)
    data[k] = v

In [ ]:
def count_class(data_list):
    ent_num = np.zeros((1000,))
    correct_num = np.zeros((1000,))
    data_num = np.zeros((1000,), dtype=np.float32)
    correct_rate = np.zeros((1000,), dtype=np.float32)

    for _, ef, lbl, cor in data_list:
        for i in ef:
            ent_num[i] += 1
        for l, c in zip(lbl, cor):
            if c:
                correct_num[l] += 1
            data_num[l] += 1
    
    data_num[data_num==0] = 1
    correct_rate = correct_num / data_num
    return ent_num, correct_num, correct_rate, data_num